In [96]:
import pandas as pd
import numpy as np
import warnings
import re
import math
warnings.filterwarnings("ignore")

In [97]:
# подгрузка всех файлов испытаний
import os
dr = 'exp'
file_names = os.listdir(dr)

In [98]:
del file_names[-3]

In [99]:
# файлы в список (список пар [имя файла, словарь датафреймов файла] )
exp_files = []
for name in file_names:
    exp_files.append([name, pd.read_excel(dr+'\\'+name, None)])
    print(name)

1квартал2017.xls
2012 1 квартал (январь, февраль, март)испр.xls
2012 2 квартал (апрель, май, июнь)испр..xls
2012 3 квартал (июль, август, сентябрь)испр..xls
2012 4 квартал (октябрь, ноябрь, декабрь)испр.xls
2013 1 квартал (январь, февраль, март)испр.xls
2013 2 квартал (апрель, май, июнь)испр..xls
2013 3 квартал (июль, август, сентябрь)испр..xls
2013 4 квартал (октябрь, ноябрь, декабрь)испр.xls
2014 1 квартал (январь, февраль, март)испр.xls
2014 2 квартал (апрель, май, июнь)испр.xls
2014 3 квартал (июль, август, сентябрь)испр.xls
2014 4 квартал (октябрь, ноябрь, декабрь)испр.xls
2015 1 квартал (январь, февраль, март)испр.xls
2015 2 квартал (апрель, май, июнь)испр.xls
2015 3 квартал ( июль, август, сентябрь)испр.xls
2015 4 квартал (октябрь, ноябрь, декабрь)испр.xls
2016 1 квартал (январь,февраль,март)испр.xls
2016 2 квартал (апрель, май, июнь)испр.xls
2016 3 квартал (июль, август, сентябрь)испр.xls
2016 4 квартал (октябрь, ноябрь, декабрь)испр.xls
2квартал2017.xls
2квартал2017_2.xlsx
3 к

In [100]:
tmp = []
for name, f in exp_files:
    for l in f:
        if l in f:
            if '№ плавки' in f[l]:
                f[l]['№ плавки'] = f[l]['№ плавки'].astype(str).str.lower()
            if '№ партии' in f[l]:
                f[l]['№ партии'] = f[l]['№ партии'].astype(str).str.lower()
    tmp.append([name, f])
exp_files = tmp

In [101]:
ls_mean = [
        'кремний (8)', 'углерод (4)', 'азот (5)', 'сера (10)', 'ванадий (13)',
        'хром (14)', 'марганец (15)', 'алюминий (7)', 'никель (18)',
        'медь (19)', 'молибден (25)', 'титан (12)', 'фосфор (9)', 'ниобий (24)',
        'бор (3)', 'водород (36)', 'кислород (37)' 
]

In [133]:
# создаем единую таблицу с химией
chemistry = exp_files[0][1][u'Хим.анализ плавок (заготовка)']

chemistry_list_names = [
    'Хим.анализ плавок (заготовка)',
    'Хим.анализ плавок(заготовка)',
    'Хим.анализ плавок (Заготовка)',
    'Хим.анализ плавок (заготовки)' 
]
s = 0
for name, exp_file in exp_files:
    for chem in chemistry_list_names:
        if chem in exp_file:
            chemistry = pd.merge(chemistry, exp_file[chem], how = 'outer')
            s += exp_file[chem].shape[0]

chemistry[u'№ плавки'] = chemistry[u'№ плавки'].astype(str)
chemistry.to_excel('chemistry.xlsx')

print(s)
print(chemistry.shape)

16057
(12306, 17)


In [134]:
def ch2f(x):
    if (type(x)==float) | (type(x)==int):
        x = str(x)
    if u'г' in x:
        print(x)
        x = x.replace(u'г.п.', '')
        x = x.replace(u'гп', '')
        x = str(x)
    if u'0.012. 0.014. 0.013' in str(x):
        x = x.replace(u'0.012. 0.014. 0.013', u'0.013')
    if x[-4:-2]=='e-':
        return float(x)
    elif '-' in str(x): 
#         print(x)
        x = x.replace(' ', '')
        x = str(x) + '0'
        tmp_map = list(map(float, str(x).replace(',', '.').split('-')))
        if (abs(tmp_map[-1]-tmp_map[-2])>0.05) | (tmp_map[-1]==0):
            return np.mean(tmp_map[:-1])
        else:
            return np.mean(tmp_map)
    else:
        return float(str(x).replace(',', '.'))

    
chem_list = [
    u'C',
    u'Mn',
    u'Si',
    u'P',
    u'S',
    u'Cr',
    u'Ni',
    u'Cu',
    u'Al',
    u'V',
    u'Ti',
    u'Nb',
    u'Mo',
    u'N',
    u'B'
]

for element in chem_list:
    chemistry[element] = chemistry[element].apply(ch2f)
chemistry.shape

0.18 г.п.
0.22 гп
0.21 г.п.


(12306, 17)

In [135]:
print(chemistry.shape)
chemistry = chemistry.drop_duplicates()
print(chemistry.shape)

(12306, 17)
(12298, 17)


In [138]:
dupl_df = chemistry[chemistry['№ плавки'].duplicated(keep = False)].sort_values(['№ плавки'])
# .to_excel('check_dupl.xlsx')
# merge_table[merge_table[['№ плавки', '№ партии']].duplicated(keep = False)].sort_values(['№ плавки'])
# count = 0
def c_dif(x):
    fl = False
    for i in x['C']:
        for j in x['C']:
            if (abs(i-j)<0.1) & (abs(i-j)>0):
                fl = True
    if fl:
         print(x[['№ плавки','C']])       
    
dupl_df.groupby('№ плавки').apply(c_dif)
# print()

In [ ]:
dupl_df

In [105]:
mode = pd.read_excel('all_modes_clean_2.xlsx')
mode.shape
mode[u'№ плавки'] = mode[u'№ плавки'].astype(str)
mode.shape

(29255, 31)

In [106]:
mode.shape
mode[mode[['№ плавки', '№ партии']].duplicated(keep = False)].sort_values('№ партии')

,№ плавки,№ партии,"НД (гост, ту)",марка стали,1 зона по ВТР закалка,1 зона уставка закалка,1 зона фактическая температура закалка,2 зона по ВТР закалка,3 зона по ВТР закалка,3 зона уставка закалка,...,3 зона ВТР и уставка отпуск,3 зона фактическая температура отпуск,4 зона ВТР и уставка отпуск,5 зона ВТР и уставка отпуск,5 зона фактическая температура отпуск,Температура трубы на выгрузке из печи отпуска,"шаг балок закалочная печь, сек","шаг балок отпускная печь, сек",диаметр,толщина стенки


In [107]:
merge_table = pd.merge(mode, chemistry, how ='left', on=['№ плавки'])
print(merge_table.shape)

(35441, 47)


In [108]:
merge_table.columns

Index(['№ плавки', '№ партии', 'НД (гост, ту)', 'марка стали',
       '1 зона по ВТР закалка', '1 зона уставка закалка',
       '1 зона фактическая температура закалка', '2 зона по ВТР закалка',
       '3 зона по ВТР закалка', '3 зона уставка закалка',
       '3 зона фактическая температура закалка',
       'Температура трубы на выгрузке из печи закалки',
       'Скорость прохождения трубы через спрейер, м/с',
       't˚ C трубы после спреера', 'Расход воды на закалочный спрейер, м3',
       'время выдержки в закалочной ванне, сек.',
       't˚ C воды в закалочной ванне', 't˚ C трубы после ванны',
       '1 зона ВТР и уставка отпуск', '1 зона фактическая температура отпуск',
       '2 зона ВТР и уставка отпуск', '3 зона ВТР и уставка отпуск',
       '3 зона фактическая температура отпуск', '4 зона ВТР и уставка отпуск',
       '5 зона ВТР и уставка отпуск', '5 зона фактическая температура отпуск',
       'Температура трубы на выгрузке из печи отпуска',
       'шаг балок закалочная печь

In [109]:
for name, f in exp_files:
    for l in chemistry_list_names:
        if l in f:
            if '№ плавки' in f[l]:
                for i in f[l]['№ плавки']:
                    if i  == 'v20694':
                        print(l, name)

Хим.анализ плавок (заготовка) 2012 1 квартал (январь, февраль, март)испр.xls
Хим.анализ плавок (заготовка) 2012 2 квартал (апрель, май, июнь)испр..xls


In [110]:
merge_table.to_excel('merge_table_new.xlsx')

In [111]:
mean_chem_steel = pd.read_excel('mean_chem_steel.xlsx')

In [112]:
merge_table = pd.read_excel('merge_table_new.xlsx')

In [113]:
merge_table.shape, mean_chem_steel.shape

((35441, 47), (64, 16))

In [114]:
merge_table[merge_table['C'].isnull()].shape

(950, 47)

In [115]:
ls = merge_table.columns[:-15]
merge_without_chem = merge_table[merge_table['C'].isnull()]
merge_without_chem = merge_without_chem[ls]

In [116]:
merge_table = merge_table[merge_table['C'].notnull()]
merge_table.shape

(34491, 47)

In [117]:
merge_mean_chem = pd.merge(merge_without_chem, mean_chem_steel, how='outer', on='марка стали')

In [118]:
merge_mean_chem.shape

(992, 47)

In [119]:
merge_table = pd.merge(merge_table, merge_mean_chem, how ='outer')
merge_table.shape

(35483, 47)

In [74]:
"""
Усредняем близкие по химии плавки, и оставляем одну из двух 
(пока первую, потом по марке можно будет прикрутить) для далеких (если разница между любыми хим элементами больше 0.01)
"""
def to_mean_or_not_to_mean(x):
    c_need_list = []
    for i in x['марка стали']:
        if (i=='Д')|(i=='ДБ'):
            c_need_list.append(float('0.45'))
        elif (type(i)!=float) & (i!='ГОСТ Р53366 "N80 Q"') & (i!='ТУ 1319-369'):
#             print(type(i), i)
            c_need = float(str(i)[0:2])/100
            c_need_list.append(c_need)
        else:
            c_need_list.append(float('nan'))
#     print(c_need_list)
    is_different = False
    for ch in chem_list:
        list_ch = []
        for i in x[ch]:
            list_ch.append(i)
        for i in x[ch]:
            for j in x[ch]:
                if (abs(i-j)>0.01)|(is_different):
                    is_different = True
#                     x[ch] = list_ch[0]
#         x[ch] = x[ch].mean()
    
    if is_different:
        list_C = []
        for c in x['C']:
            list_C.append(c)
        index = -1
        min_dif = 0.04
        for i in range(0,len(list_C)):
            for c_n in c_need_list:
                if abs(c_n-list_C[i])<min_dif:
                    min_dif=abs(c_n-list_C[i])
                    index = i
        for ch in chem_list:
            list_ch = []
            for i in x[ch]:
                list_ch.append(i)
            for i in x[ch]:
                for j in x[ch]:
                    if (abs(i-j)>0.01)|(is_different):
                        is_different = True
                        x[ch] = list_ch[index]
            x[ch] = x[ch].mean()
    else:
        x[ch] = x[ch].mean()
    return x

def get_mean(df):
    df = df.groupby('№ плавки').apply(to_mean_or_not_to_mean)
    df = df.reset_index(drop=True)
    return df

In [130]:
# merge_table[merge_table[['№ плавки', '№ партии']].duplicated(keep = False)].sort_values(['№ плавки'])
# count = 0
def c_dif(x):
    fl = False
    for i in x['C']:
        for j in x['C']:
            if (abs(i-j)<0.1) & (abs(i-j)>0):
                fl = True
    if fl:
         print(x[['№ плавки','C']])       
    
merge_table.groupby('№ плавки').apply(c_dif)
# print()

      № плавки        C
34591   025690  0.28349
34592   025690  0.28349
34822   025690  0.28363
34823   025690  0.28363
      № плавки        C
34593   026572  0.28349
34825   026572  0.28363
      № плавки        C
34594   042520  0.28349
34595   042520  0.28349
34832   042520  0.28363
  № плавки         C
2    10043  0.280000
3    10043  0.291385
4    10043  0.280000
5    10043  0.291385
   № плавки     C
10   110158  0.20
11   110158  0.29
   № плавки      C
18   110642  0.212
19   110642  0.190
20   110642  0.100
   № плавки      C
21   110644  0.200
22   110644  0.224
   № плавки      C
32   111525  0.272
33   111525  0.275
34   111525  0.272
35   111525  0.275
   № плавки       C
37   111578  0.2730
38   111578  0.2835
39   111578  0.2730
40   111578  0.2835
41   111578  0.2730
42   111578  0.2835
   № плавки       C
43   111584  0.2820
44   111584  0.2856
45   111584  0.2820
46   111584  0.2856
   № плавки       C
97   116000  0.2800
98   116000  0.2955
    № плавки      C
100  

      № плавки     C
18271   5f2854  0.27
18272   5f2854  0.29
18273   5f2854  0.27
18274   5f2854  0.29
18275   5f2854  0.27
18276   5f2854  0.29
      № плавки        C
35012   611084  0.19743
35349   611084  0.20063
      № плавки     C
21887   621982  0.18
21888   621982  0.08
21889   621982  0.18
21890   621982  0.08
      № плавки     C
21899   622582  0.18
21900   622582  0.08
      № плавки     C
21942   624152  0.20
21943   624152  0.19
21944   624152  0.20
21945   624152  0.19
21946   624152  0.20
21947   624152  0.19
      № плавки     C
22008   630731  0.18
22009   630731  0.09
      № плавки     C
22010   630732  0.11
22011   630732  0.19
      № плавки        C
35014   631360  0.19743
35015   631360  0.19743
35351   631360  0.20063
35352   631360  0.20063
      № плавки        C
35016   6f0774  0.19743
35266   6f0774  0.19724
      № плавки        C
25290   6f5213  0.35000
25291   6f5213  0.35000
34956   6f5213  0.36306
34957   6f5213  0.36306
      № плавки      C
27176 

""


In [76]:
merge_table['C'] .describe()

count    35448.000000
mean         0.267297
std          0.074888
min          0.030000
25%          0.220000
50%          0.280000
75%          0.300000
max          1.020000
Name: C, dtype: float64

In [77]:
def switch_c_mn(x):
    if x['C']>=0.9:
        tmp = x['C']
        x['C']= x['Mn']
        x['Mn'] = tmp
        print(x['C'], x['Mn'])
    return x
merge_table = merge_table.apply(switch_c_mn, axis =1)

0.29 1.0
0.29 1.0
0.29 1.0
0.29 1.0
0.36 1.0
0.3 0.98
0.27 1.0
0.27 1.0
0.27 1.02
0.27 1.02
0.27 1.02


In [78]:
merge_table['C'].describe()
# .value_counts().sort_index()

count    35448.000000
mean         0.267075
std          0.073758
min          0.030000
25%          0.220000
50%          0.280000
75%          0.300000
max          0.470000
Name: C, dtype: float64

In [79]:
print(merge_table.shape)
merge_table = get_mean(merge_table)
print(merge_table.shape)

(35483, 47)
(35483, 47)


In [80]:
merge_table.drop_duplicates(chem_list+['№ плавки', '№ партии'], inplace=True)
merge_table.shape

(32789, 47)

In [132]:
merge_table[merge_table[['№ плавки', '№ партии']].duplicated(keep = False)].sort_values(['№ плавки']).to_excel('check_dupl.xlsx')

In [82]:
merge_table.shape

(32789, 47)

In [83]:
merge_table.drop_duplicates(['№ плавки', '№ партии'], inplace=True)
merge_table.shape

(29244, 47)

In [84]:
# merge_table = pd.read_excel('merge_table_mean_chem.xlsx')

In [94]:
copy = merge_table[['марка стали']+chem_list+['№ плавки', '№ партии']]

In [86]:
merge_table.shape

(29244, 47)

In [87]:
merge_table = pd.concat([merge_table[merge_table['Al']<0.08], merge_table[merge_table['Al'].isnull()]])
merge_table.reset_index(drop=True, inplace=True)

In [88]:
copy[copy['№ плавки']=='434396']

,марка стали,C,Mn,Si,P,S,Cr,Ni,Cu,Al,V,Ti,Nb,Mo,N,B,№ плавки,№ партии
11719,13ХФА,0.137,0.51,0.31,0.009,0.002,0.57,0.048,0.014,0.038,0.058,0.002,0.005,0.005,0.0036,NaN,434396,408104


In [89]:
copy = copy[copy['B']>0.0005]
print(copy.shape)
copy['марка стали'].unique()
# copy

(289, 18)


array(['13ХФА', '38Г2С', '20А', '20А-1', '09Г2С', '20'], dtype=object)

In [93]:
merge_table.shape

(29237, 47)

In [134]:
merge_table.to_excel('merge_table_mean_chem.xlsx')

In [95]:
copy.to_excel('check_chem.xlsx')